In [3]:
import xarray as xr
import glob
import numpy as np

# Directory containing the NetCDF files
data_dir = "C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050/"
file_pattern = f"{data_dir}*.nc"

# Step 1: Find and sort all NetCDF files
file_list = sorted(glob.glob(file_pattern))  # Sort to ensure chronological order
print(f"Found {len(file_list)} files: {file_list}")

Found 17 files: ['C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18500101-18591231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18600101-18691231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18700101-18791231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18800101-18891231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18900101-18991231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.19000101-19091231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE

In [4]:
import xarray as xr
import glob
import numpy as np

# Directory containing the NetCDF files
data_dir = "C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050/"
file_pattern = f"{data_dir}*.nc"

# Step 1: Find and sort all NetCDF files
file_list = sorted(glob.glob(file_pattern))  # Sort to ensure chronological order
print(f"Found {len(file_list)} files: {file_list}")

# Define NAO region coordinates
lat_min, lat_max = 20.2617801, 80.57591623  # 20°N to 80°N
lon_min_360, lon_max_360 = 270, 40  # 90°W (270°E) to 40°E

# Step 2: Process each file
processed_datasets = []
for file in file_list:
    # Open the dataset
    ds = xr.open_dataset(file, decode_times=False)

    # Select the two parts of the longitude range
    lon_part1 = ds.sel(
        lat=slice(lat_min, lat_max),  # 20°N to 80°N
        lon=slice(0, lon_max_360)     # 0° to 40°E
    )
    lon_part2 = ds.sel(
        lat=slice(lat_min, lat_max),  # 20°N to 80°N
        lon=slice(lon_min_360, 358.75)  # 270°E to 358.75°
    )

    # Concatenate the two longitude parts
    nao_data = xr.concat([lon_part1, lon_part2], dim='lon')

    # Rename coordinates
    nao_data = nao_data.rename({'lon': 'longitude', 'lat': 'latitude'})

    # Decode the time coordinate
    nao_data['time'] = xr.decode_cf(nao_data)['time']

    # Select winter months (November, December, January, February, March)
    winter_months = [11, 12, 1, 2, 3]
    nao_data = nao_data.sel(time=nao_data['time'].dt.month.isin(winter_months))

    # Select only the required variables
    nao_data = nao_data[['time', 'longitude', 'latitude', 'V050']]

    # Append to list of processed datasets
    processed_datasets.append(nao_data)


Found 17 files: ['C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18500101-18591231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18600101-18691231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18700101-18791231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18800101-18891231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.18900101-18991231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE2-1251.012.cam.h1.V050.19000101-19091231.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050\\b.e21.BHISTsmbb.f09_g17.LE

In [5]:
# Step 3: Concatenate all processed datasets along the time dimension
merged_nao_data = xr.concat(processed_datasets, dim='time', data_vars='all', coords='different')


In [6]:
print(merged_nao_data)

<xarray.Dataset> Size: 670MB
Dimensions:    (time: 24916, latitude: 64, longitude: 105)
Coordinates:
  * time       (time) object 199kB 1850-01-01 00:00:00 ... 2015-01-01 00:00:00
  * longitude  (longitude) float64 840B 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * latitude   (latitude) float64 512B 20.26 21.2 22.15 ... 77.75 78.69 79.63
Data variables:
    V050       (time, latitude, longitude) float32 670MB -3.093 ... -10.11
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1251.012
    logname:           sunseon
    host:              mom1
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1251-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1


In [7]:
# Sort by time to ensure chronological order
merged_nao_data = merged_nao_data.sortby('time')

# Step 4: Verify the merged dataset
print("Merged dataset shape:", merged_nao_data.dims)
print("Time range:", merged_nao_data['time'].min().values, "to", merged_nao_data['time'].max().values)
print("V050 shape:", merged_nao_data['V050'].shape)

# Step 5: Save the merged dataset
output_file = "C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050/merged_cesm2_V050_testset.nc"
merged_nao_data.to_netcdf(output_file)
print(f"Saved merged dataset to {output_file}")
# Step 6: Clean up
for ds in processed_datasets:
    ds.close()  # Close each dataset to free up resources

Merged dataset shape: FrozenMappingWarningOnValuesAccess({'time': 24916, 'latitude': 64, 'longitude': 105})
Time range: 1850-01-01 00:00:00 to 2015-01-01 00:00:00
V050 shape: (24916, 64, 105)
Saved merged dataset to C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/test set V050/merged_cesm2_V050_testset.nc


In [8]:
# Step 6: Clean up
for ds in processed_datasets:
    ds.close()  # Close each dataset to free up resources

In [9]:
print(merged_nao_data['V050'])

<xarray.DataArray 'V050' (time: 24916, latitude: 64, longitude: 105)> Size: 670MB
array([[[ -3.092682  ,  -3.343086  ,  -3.4936368 , ...,  -1.3413198 ,
          -2.1791801 ,  -2.731402  ],
        [ -4.186254  ,  -4.502462  ,  -4.73709   , ...,  -2.298499  ,
          -3.164488  ,  -3.7494946 ],
        [ -5.250101  ,  -5.7450557 ,  -6.1170416 , ...,  -3.043378  ,
          -3.9489408 ,  -4.6386733 ],
        ...,
        [-11.640004  , -11.269746  , -10.9097595 , ..., -12.81273   ,
         -12.409102  , -12.019446  ],
        [-12.219515  , -11.846014  , -11.481465  , ..., -13.38257   ,
         -12.986423  , -12.599665  ],
        [-12.459867  , -12.066476  , -11.6840105 , ..., -13.691616  ,
         -13.272053  , -12.861939  ]],

       [[ -3.6706355 ,  -4.0258784 ,  -4.2998204 , ...,  -2.2401905 ,
          -2.7936058 ,  -3.2564955 ],
        [ -4.4320974 ,  -4.786875  ,  -5.071417  , ...,  -2.9060924 ,
          -3.5228405 ,  -4.016078  ],
        [ -5.1142945 ,  -5.454622  ,  -